In [1]:
import nltk
import numpy as np
import json
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

In [2]:
stemmer=PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence,words):
    sentence_words=[stem(word) for word in tokenized_sentence]
    bag=np.zeros(len(words),dtype=np.float32)
    
    for idx,w in enumerate(words):
        if w in sentence_words:
            bag[idx]=1
     
    return bag

In [3]:
with open('intents.json','r') as f:
    intents=json.load(f)
    

all_words=[]
tags=[]
xy=[]

for intent in intents['intents']:
    tag=intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w=tokenize(pattern)
        all_words.extend(w)
        xy.append((w,tag))

ignore_words=['?','!','.',',']
all_words=[stem(w) for w in all_words if w not in ignore_words]

all_words=sorted(set(all_words))
tags=sorted(tags)

#print(all_words)
# print(tags)

x_train=[]
y_train=[]

for (pattern_sentence,tag) in xy:
    bag=bag_of_words(pattern_sentence,all_words)
    x_train.append(bag)
    label=tags.index(tag)
    y_train.append(label)
    
   
x_train=np.array(x_train)
y_train=np.array(y_train)

# print(x_train)
# print(y_train)

In [4]:
#model
import torch
import torch.nn as nn

class neuralnet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(neuralnet,self).__init__()
        self.l1=nn.Linear(input_size,hidden_size)
        self.l2=nn.Linear(hidden_size,hidden_size)
        self.l3=nn.Linear(hidden_size,num_classes)
        self.relu=nn.ReLU()
        
    def forward(self,x):
        out=self.l1(x)
        out=self.relu(out)
        out=self.l2(out)
        out=self.relu(out)
        out=self.l3(out)
        
        return out
    

In [5]:
class chatdataset(Dataset):
    def __init__(self):
        self.n_samples=len(x_train)
        self.x_data=x_train
        self.y_data=y_train
        
    def __getitem__(self,idx):
        return self.x_data[idx],self.y_data[idx]
    
    def __len__(self):
        return self.n_samples
    
    

batch_size=8
num_epoch=1000
learning_rate=0.001
input_size=len(x_train[0])
hidden_size=8
output_size=len(tags)


dataset=chatdataset()
train_loader=DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,num_workers=0)


In [6]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=neuralnet(input_size,hidden_size,output_size).to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epoch):
    for (words,labels) in train_loader:
        words=words.to(device)
        labels=labels.to(dtype=torch.long).to(device)
        
        #forward
        outputs=model(words)
        loss=criterion(outputs,labels)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epoch}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')  
        

Epoch [100/1000], Loss: 1.3474
Epoch [200/1000], Loss: 0.1456
Epoch [300/1000], Loss: 0.0380
Epoch [400/1000], Loss: 0.0389
Epoch [500/1000], Loss: 0.0088
Epoch [600/1000], Loss: 0.0015
Epoch [700/1000], Loss: 0.0055
Epoch [800/1000], Loss: 0.0014
Epoch [900/1000], Loss: 0.0007
Epoch [1000/1000], Loss: 0.0014
final loss: 0.0014


In [7]:
model_state=model.state_dict()

In [8]:
import random

In [9]:
mm=neuralnet(input_size,hidden_size,output_size).to(device)
mm.load_state_dict(model_state)
mm.eval()

neuralnet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [11]:
bot_name="Alice"
print("Let's chat! type 'quit' to exit")
while True:
    sentence=input('You:')
    if(sentence == "quit"):
        break
        
    sentence=tokenize(sentence)
    x=bag_of_words(sentence,all_words)
    x=x.reshape(1,x.shape[0])
    x=torch.from_numpy(x).to(device)
    
    output=mm(x)
    #value,index
    _,predicted=torch.max(output,dim=1)
    tag = tags[predicted.item()]
    
    probs=torch.softmax(output,dim=1)
    prob=probs[0][predicted.item()]
    
    if prob.item()>0.75:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                print(f'{bot_name}: {random.choice(intent["responses"])}')
    else:
        print(f'{bot_name}:I do not understand')

Let's chat! type 'quit' to exit
You:hi alice
Alice: Hello, thanks for visiting
You:how are you
Alice: Hi there, how can I help?
You:which item you have
Alice: We have coffee and tea
You:what is the price
Alice: Have a nice day
You:good bye
Alice: Have a nice day
You:quit
